# Chat Agent Executor

In [ ]:
!pip install --quite -U langchain langchain_community tavily-python dotenv

## Create Langchain agent

In [24]:
from langchain import hub
from langchain_community.chat_models.ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_core.runnables import RunnableConfig

In [25]:
# Env
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
TAVILY_API_KEY = os.environ.get("TAVILY_API_KEY")

In [26]:
# Choose LLM
llm = ChatOllama(model="llama2", streaming=True)

# Tools
tools = [TavilySearchResults(tavily_api_key=TAVILY_API_KEY, max_results=1)]

# Prompt
prompt = hub.pull("hwchase17/react")

In [27]:
agent_runnable = create_react_agent(llm, tools, prompt)

## Define the graph state

In [28]:
from typing import TypedDict, Annotated, Sequence
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator

In [29]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

## Define the nodes

Define few different nodes and edges in our graph. A node can be eithher a function or a runnable.

Here we need 2 types of nodes:
1. The agent: responsible for deciding what actions to take
2. A function to invoke tools: if the agent decides to take an action, this node will then execute that action.

Edges could be conditional based on the output of node.
1. Conditional Edge: after the agent is called, we should either:
   a. If the agent said to take an action, then the function to invoke toools should be called.
   b. If the agent said that it was finisied, then it should finish.
2. Normal Edge: after the tools are invoked, it should always go back to the agent to decide wwhat to do next

In [30]:
from langchain_core.messages import FunctionMessage
from langgraph.prebuilt import ToolInvocation, ToolExecutor
import json

# Helper class to execute tools
tool_executor = ToolExecutor(tools)

# Define logic that will be used to determine which conditional edge to go down
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    else:
        return "continue"

def call_model(state):
    # Note - Logic like only pass the 5 most recent message to model etc, can be added here
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

def call_tool(state):
    messages = state["messages"]
    last_message = messages[-1]

    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"])
    )

    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content=str(response), name=action.tool)
    return {"messages": [function_message]}

## Define the graph

Put all together and define the graph

In [31]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END
    }
)

workflow.add_edge("action", "agent")

app = workflow.compile()

In [33]:
from langchain_core.messages import HumanMessage
inputs = {"messages": [HumanMessage(content="what is the weather in SF")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the weather in SF'),
  AIMessage(content="\nThe current weather in San Francisco, California is:\n\n* Temperature: 62°F (17°C)\n* Humidity: 60%\n* Wind: NW at 5 mph\n* Conditions: Mostly Cloudy\n* Forecast: A mix of sun and cloud with a chance of light rain showers throughout the day. High near 64°F (18°C) and low around 52°F (11°C).\n\nPlease note that this information is subject to change and may not be up-to-date or accurate. It's always a good idea to check the latest weather forecast before planning your day.", response_metadata={'model': 'llama2', 'created_at': '2024-04-20T18:10:42.318812Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 2995407875, 'load_duration': 3124416, 'prompt_eval_count': 11, 'prompt_eval_duration': 267480000, 'eval_count': 152, 'eval_duration': 2722552000}, id='run-782ea381-5c1a-4c26-a0e9-6ecdd4a4f0bf-0')]}